# Use of Relu funtion as the activation function for hidden layer

In [1]:
import math
import struct as st
import numpy as np
import matplotlib.pyplot as plt

In [2]:
 # Initialize the Parameters   
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters    

In [3]:
#Activation Funtion
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    return A

def tanh(Z):               # Use of tanh function
    A=np.tanh(Z)
    return A

In [4]:
#Calculation of Z = W*A_previouse + B
def linear_forward(A,W,b):
    Z=np.dot(W,A)+b
    return Z

In [5]:
# Calculation the of the activation Funtion. 
#        A = Sigmoid(Z) **Need to Update**
def linear_activation_forward(A_prev, W, b, activation_type): 
    
    if activation_type == "Sigmoid":            #Use of activation funtion in hidden layer is Sigmoid funtion
        Z = linear_forward(A_prev,W,b)
        A = sigmoid(Z)
    elif activation_type == "Relu":
        Z = linear_forward(A_prev,W,b)
        for i in range(0,len(Z)):
            if Z < 0:
                A = 0.01*Z
            elif Z >= 0:
                A = Z
                   
    return A,Z  

In [6]:
#Compute Cost of the overall model.
#       Cost = Summation(Loss)/(Number of input data), where Loss= - Y*ln(Activation) - (1-Y)*ln(1 - Activation)
def compute_cost(AL,Y):
    data_length = Y.shape[1]
    cost =(-1/data_length) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    cost = np.squeeze(cost)
    assert(cost.shape == ())  
    return cost

In [7]:
# Linear_backward
def linear_backward(dZ, A):
    data_length = A.shape[1]
    dW = (1/data_length)*np.dot(dZ,A.T)
    db = (1/data_length)*np.sum(dZ, axis=1, keepdims=True)
    return dW,db

In [8]:
# Linear_activation_backward
def linear_activation_backward(A, cache, Layer):
    if Layer == "Output Layer":
        Y, AL = cache
        dZ2 = A - Y
        dW, db = linear_backward(dZ2, AL)
        return dZ2, dW, db
    elif Layer == "Hidden Layer":
        X, W2, dZ2 = cache
        for i range(0,len(A)):
            if A[i] >= 0:
                dZ1[i] = np.dot(W2.T,dZ2)*1
            elif A < 0:
                dZ1 = np.dot(W2.T,dZ2)*0.01
        dW, db = linear_backward(dZ1, X)
        return dZ1,dW, db

In [9]:
# Updates the parameters according to the obtained gradient values
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters)//2           # number of layers in the neural network
    
    for l in range(L):                  # Update rule for each parameter. Use a for loop.
        parameters["W" + str(l+1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]    
    return parameters

In [10]:
# Main body of the Nueral Net Model which contains the procedural instructions to the execution of Nueral net model
def L_layer_network(X,Y,learningrate,layer_dims,num_iterations,print_cost):
    costs=[]
    
    #Initailise and fetch the weights through dictionary
    parameters = initialize_parameters(layer_dims)
    L = len(parameters) // 2                           # number of layers in the neural network
    activation = {}
    linearsum = {}
    
    for i in range(0, num_iterations):
        A = X
        # Forward propagation
        for l in range(1, L):
            A_prev = A
            activation['A'+str(l)], linearsum['Z'+str(l)] = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b' + str(l)], "Relu")   #Calculation of activaion output of hidden layer
            A = activation['A'+str(l)]
        
        activation['A'+str(L)], linearsum['Z'+str(L)] = linear_activation_forward(A, parameters['W'+str(L)], parameters['b' + str(L)], "Sigmoid")        #Calculation of activation output of output layer
    
        # Compute cost
        cost = compute_cost(activation['A'+str(L)], Y)
        costs.append(cost)
        
         # Stores the gradient values in the dictionary grads
        grads = {}
        data_length = Y.shape[1]
        
        # Backward propagation
        activation['A'+str(0)] = X
        cache = (Y, activation['A'+str(L-1)])
        linearsum['dZ'+str(L)] ,grads['dW'+str(L)], grads['db'+str(L)] = linear_activation_backward(activation['A'+str(L)], cache, "Output Layer")
        
        for l in reversed(range(1,L)):
            cache = (activation['A'+str(l-1)], parameters['W'+str(l+1)], linearsum['dZ'+str(l+1)])
            linearsum['dZ'+str(l)], grads['dW'+str(l)], grads['db'+str(l)] = linear_activation_backward(activation['A'+str(l)], cache, "Hidden Layer")
        
    
        # Update the parameters
        parameters =  update_parameters(parameters, grads, learningrate)
        
        #Vary the learning rate
        if(cost > 0.37):
            learningrate = learningrate
        elif(cost<0.37 and cost>0.2):
            learningrate = 0.5
        elif(cost<0.25 and cost>0.1):
            learningrate = 0.3
            
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
    
    #Plot the cost funtion curve
    plt.plot(costs)
    plt.ylabel("Cost")
    plt.xlabel("iterations")
    plt.show()
    return parameters,activation,linearsum

In [11]:
# Input Data(MNIST)

filename = {'images' : 'E:/Handwritten digit recognisation/dataset/train-images-idx3-ubyte' ,'labels' : 'E:/Handwritten digit recognisation/dataset/train-labels-idx1-ubyte'}
train_imagesfile = open(filename['images'],'rb')

MagicNumber = st.unpack('>I',train_imagesfile.read(4))[0]
nImg = st.unpack('>I',train_imagesfile.read(4))[0] #num of images
nR = st.unpack('>I',train_imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',train_imagesfile.read(4))[0] #num of column
print("Magic Number : ", MagicNumber)
print("num of images : ", nImg)
print("num of rows : ", nR)
print("num of column : ", nC)

images_array = np.zeros((nImg,nR,nC))

nBytesTotal = nImg*nR*nC*1 #since each pixel data is 1 byte
images_array = np.asarray(st.unpack('>'+'B'*nBytesTotal,train_imagesfile.read(nBytesTotal))).reshape((nImg,nR,nC))

X = (1/255)*images_array.reshape(nImg,nR*nC)

print("Size images_array :",X.shape)

Magic Number :  2051
num of images :  60000
num of rows :  28
num of column :  28
Size images_array : (60000, 784)


In [12]:
layer_dims = [np.squeeze(np.shape(X[0])),20,10,10]
learningrate = 1
print_cost=True

#Label Data(MNIST)
train_labelfile = open(filename['labels'],'rb')
MagicNumber = st.unpack('>I',train_labelfile.read(4))[0]
nlabel = st.unpack('>I',train_labelfile.read(4))[0] #num of labels
print("Magic Number : ", MagicNumber)
print("num of images : ", nlabel)
label_array = np.zeros((nlabel))

nBytesTotal = nlabel*1 #since each pixel data is 1 byte
label_array = np.asarray(st.unpack('>'+'B'*nBytesTotal,train_labelfile.read(nBytesTotal))).reshape((nlabel))

print("Size images_array : ",len(label_array))

Y=np.zeros((len(label_array),10))
for i in range (0,len(label_array)):
    for j in range (0,10):
        if label_array[i]==j:
            Y[i][j] = 1
        else:
            Y[i][j] = 0

parameters,activation,linearsum = L_layer_network(X.T, Y.T, learningrate, layer_dims, 5000, print_cost)

Magic Number :  2049
num of images :  60000
Size images_array :  60000


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Decodes the output that the trained model predicts on the testing set
def Output(A2):
    key = A2[0][0]
    p = 0
    for i in range (0,10):
        if A2[i][0]>=key:
                p = i
                key = A2[i][0]
    return p

In [ ]:
# Calculates the accuracy the trained model is able to achieve
def accuracy_predictor(out, label_array):
    fav = 0
    for i in range(0,51):
        if(out[i]==label_array[i]):
            fav = fav + 1
    accuracy = fav/51*100
    return accuracy

In [ ]:
def modeltest(X,parameters,activation,linearsum,out):
    L = len(parameters) // 2 
    A = X
    for l in range(1, L):
            A_prev = A
            activation['A'+str(l)], linearsum['Z'+str(l)] = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b' + str(l)], "tanh")   #Calculation of activaion output of hidden layer
            A = activation['A'+str(l)]
        
    activation['A'+str(L)], linearsum['Z'+str(L)] = linear_activation_forward(A, parameters['W'+str(L)], parameters['b' + str(L)], "Sigmoid")        #Calculation of activation output of output layer
    
    A = Output(activation['A'+str(L)])
    out.append(A)
    return out

In [ ]:
Train_X=np.zeros(shape=(784,1))
out = []
filename = {'images' : 'E:/Handwritten digit recognisation/dataset/t10k-images-idx3-ubyte' ,'labels' : 'E:/Handwritten digit recognisation/dataset/t10k-labels-idx1-ubyte'}
Test_imagesfile = open(filename['images'],'rb')

MagicNumber = st.unpack('>I',Test_imagesfile.read(4))[0]
nImg = st.unpack('>I',Test_imagesfile.read(4))[0] #num of images
nR = st.unpack('>I',Test_imagesfile.read(4))[0] #num of rows
nC = st.unpack('>I',Test_imagesfile.read(4))[0] #num of column
print("Magic Number : ", MagicNumber)
print("num of images : ", nImg)
print("num of rows : ", nR)
print("num of column : ", nC)

Tex = np.zeros((nImg,nR,nC))

nBytesTotal = nImg*nR*nC*1 #since each pixel data is 1 byte
Tex = (1/255)*np.asarray(st.unpack('>'+'B'*nBytesTotal,Test_imagesfile.read(nBytesTotal))).reshape((nImg,nR*nC))

print("Size images_array : ",Tex.shape)

Test_labelfile = open(filename['labels'],'rb')

MagicNumber = st.unpack('>I',Test_labelfile.read(4))[0]
nlabel = st.unpack('>I',Test_labelfile.read(4))[0] #num of labels
print("Magic Number : ", MagicNumber)
print("num of images : ", nlabel)

label_arrayT = np.zeros((nlabel))

nBytesTotal = nlabel*1 #since each pixel data is 1 byte
label_arrayT= np.asarray(st.unpack('>'+'B'*nBytesTotal,Test_labelfile.read(nBytesTotal))).reshape((nlabel))


print("Size images_array : ",len(label_arrayT))

In [ ]:
for j in range(0,len(label_arrayT)):
    for i in range(0,784):
        Train_X[i][0] = Tex[j][i]
    out = modeltest(Train_X, parameters,activation,linearsum, out)
    
accuracy = accuracy_predictor(out,label_arrayT)    
print("A :",out, end=" ")
print("\n\naccuracy = ", accuracy, end="%")
print("\nout",len(out))